In [1]:
# Weight Decay 权重衰减

In [2]:
# 控制模型容量
# 限制参数范围或者数量
# 通常不限制偏移b
# 平方范数作为柔性限制
# 限制参数范围的方法
# 自己大小权重先缩小 权重衰退 再减学习率*梯度

In [4]:
%matplotlib inline
import torch
from torch import nn
from d2l import torch as d2l

In [5]:
n_train, n_test, num_inputs, batch_size = 20, 100, 200, 5
true_w, true_b = torch.ones((num_inputs, 1)) * 0.01, 0.05
train_data = d2l.synthetic_data(true_w, true_b, n_train)
train_iter = d2l.load_array(train_data, batch_size)
test_data = d2l.synthetic_data(true_w, true_b, n_test)
test_iter = d2l.load_array(test_data, batch_size, is_train=False)


In [6]:
def init_params():
    w = torch.normal(0, 1, size=(num_inputs, 1), requires_grad=True)
    b = torch.zeros(1, requires_grad=True)
    return [w, b]


In [ ]:
def l2_penalty(w):
    return torch.sum(w.pow(2)) / 2
